In [7]:
conda install openpyxl

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import glob
import openpyxl

In [54]:
rebelde =pd.read_csv('/Users/ctalamilla/Desktop/TacaTaca/data/Eólica 2/csv/20682654.csv', skiprows=0, sep='\t').set_axis(['fecha','hora','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
rebelde

,fecha,hora,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20,08:21:07,2.412,3.600,33.7
1,03/13/20,08:22:07,1.188,2.412,35.1
2,03/13/20,08:23:07,0.000,1.188,35.1
3,03/13/20,08:24:07,0.000,0.000,35.1
4,03/13/20,08:25:07,0.000,1.188,35.1
...,...,...,...,...,...
59417,04/23/20,14:38:07,16.848,20.448,332.7
59418,04/23/20,14:39:07,15.624,20.448,332.7
59419,04/23/20,14:40:07,18.036,22.860,351.0
59420,04/23/20,14:41:07,10.836,16.848,338.3


In [55]:
rebelde['fecha'] = rebelde['fecha']+ ' ' + rebelde['hora']
rebelde

,fecha,hora,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:21:07,08:21:07,2.412,3.600,33.7
1,03/13/20 08:22:07,08:22:07,1.188,2.412,35.1
2,03/13/20 08:23:07,08:23:07,0.000,1.188,35.1
3,03/13/20 08:24:07,08:24:07,0.000,0.000,35.1
4,03/13/20 08:25:07,08:25:07,0.000,1.188,35.1
...,...,...,...,...,...
59417,04/23/20 14:38:07,14:38:07,16.848,20.448,332.7
59418,04/23/20 14:39:07,14:39:07,15.624,20.448,332.7
59419,04/23/20 14:40:07,14:40:07,18.036,22.860,351.0
59420,04/23/20 14:41:07,14:41:07,10.836,16.848,338.3


In [56]:
rebelde = rebelde.drop(['hora'], axis=1)
rebelde

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:21:07,2.412,3.600,33.7
1,03/13/20 08:22:07,1.188,2.412,35.1
2,03/13/20 08:23:07,0.000,1.188,35.1
3,03/13/20 08:24:07,0.000,0.000,35.1
4,03/13/20 08:25:07,0.000,1.188,35.1
...,...,...,...,...
59417,04/23/20 14:38:07,16.848,20.448,332.7
59418,04/23/20 14:39:07,15.624,20.448,332.7
59419,04/23/20 14:40:07,18.036,22.860,351.0
59420,04/23/20 14:41:07,10.836,16.848,338.3


In [57]:
rebelde['fecha'] = pd.to_datetime(rebelde.fecha)
rebelde

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,2020-03-13 08:21:07,2.412,3.600,33.7
1,2020-03-13 08:22:07,1.188,2.412,35.1
2,2020-03-13 08:23:07,0.000,1.188,35.1
3,2020-03-13 08:24:07,0.000,0.000,35.1
4,2020-03-13 08:25:07,0.000,1.188,35.1
...,...,...,...,...
59417,2020-04-23 14:38:07,16.848,20.448,332.7
59418,2020-04-23 14:39:07,15.624,20.448,332.7
59419,2020-04-23 14:40:07,18.036,22.860,351.0
59420,2020-04-23 14:41:07,10.836,16.848,338.3


In [52]:
rebelde.to_csv('../data/Eólica 2/all_files/20682654_fix.csv', sep=',')

In [11]:
archivosAll = np.array(glob.glob('../data/Eólica 1/all_files/*'))
rutas = pd.DataFrame({'files': archivosAll})
rutas

,files
0,../data/Eólica 1/all_files/E1_22-10-2021.csv
1,../data/Eólica 1/all_files/E1_20-04-22.csv
2,../data/Eólica 1/all_files/20682654.xlsx
3,../data/Eólica 1/all_files/E1_17-03-2022.csv
4,../data/Eólica 1/all_files/E1_29-03-2022.csv
5,../data/Eólica 1/all_files/E1_17-03-2022.xlsx
6,../data/Eólica 1/all_files/E1_14-12-2021.csv
7,../data/Eólica 1/all_files/20682650.csv
8,../data/Eólica 1/all_files/20682654.csv
9,../data/Eólica 1/all_files/E1_22-10-2021.xlsx


In [74]:
def cleanEolica(path, estacion):
        archivosAll = np.array(glob.glob(path +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})


        eolica1 = []
        for index, value in rutas.files.str.contains('csv').items():
            if (value== True):
                #print(rutas.files[index])
                file_readed = pd.read_csv(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
                file_readed['fecha'] = pd.to_datetime(file_readed['fecha'] )
                file_readed['estacion']= estacion
                eolica1.append(file_readed)
                #print(file_readed)
            else:
                #print(rutas.files[index])
                file_readed = pd.read_excel(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
                file_readed['fecha'] = pd.to_datetime(file_readed['fecha'] )
                file_readed['estacion']= estacion
                eolica1.append(file_readed)
                #print(file_readed)

        return pd.concat(eolica1).drop_duplicates(['fecha']).reset_index(drop=True)

In [76]:
EstacionEolica1 = cleanEolica('../data/Eólica 1/all_files', 'Eolica1')
EstacionEolica1

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,estacion
0,2021-06-08 16:34:05,28.872,42.084,206.4,Eolica1
1,2021-06-08 16:35:05,32.472,39.672,210.6,Eolica1
2,2021-06-08 16:36:05,27.648,36.072,226.0,Eolica1
3,2021-06-08 16:37:05,30.060,38.484,219.0,Eolica1
4,2021-06-08 16:38:05,24.048,27.648,219.0,Eolica1
...,...,...,...,...,...
336539,2021-12-12 23:55:00,32.472,37.260,220.4,Eolica1
336540,2021-12-12 23:56:00,34.848,39.672,220.4,Eolica1
336541,2021-12-12 23:57:00,31.248,34.848,227.4,Eolica1
336542,2021-12-12 23:58:00,27.648,31.248,223.2,Eolica1


In [78]:
EstacionEolica1.duplicated(['fecha']).sum()

0

In [77]:
EstacionEolica2 = cleanEolica('../data/Eólica 2/all_files', 'Eolica2')
EstacionEolica2

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,estacion
0,2022-03-17 16:48:48,30.060,43.272,158.6,Eolica2
1,2022-03-17 16:53:48,31.248,49.284,160.0,Eolica2
2,2022-03-17 16:58:48,32.472,46.872,165.7,Eolica2
3,2022-03-17 17:03:48,32.472,48.096,162.9,Eolica2
4,2022-03-17 17:08:48,15.624,24.048,162.9,Eolica2
...,...,...,...,...,...
241448,2022-01-07 07:58:27,-888.880,-888.880,-888.9,Eolica2
241449,2022-01-07 07:59:27,-888.880,-888.880,-888.9,Eolica2
241450,2022-01-07 08:00:27,-888.880,-888.880,-888.9,Eolica2
241451,2022-01-07 08:01:27,-888.880,-888.880,-888.9,Eolica2


In [79]:
EstacionEolica2.duplicated(['fecha']).sum()

0

In [81]:
EstacionEolica1.to_csv('Eolica1_cleaned.csv', sep=',')
EstacionEolica2.to_csv('Eolica2_cleaned.csv', sep=',')